# DAAN 888 Project

In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
Dataset_1 = pd.read_csv(r'C:\Users\burq_\Downloads\DAAN 888-Design and Implementation of Analytics system\DAAN 888 Datasets\Final datasets\Dataset-1-owid-covid-data.csv')

Dataset_1.drop(Dataset_1.columns[[0]], axis=1, inplace=True)

list(Dataset_1.columns)

['continent',
 'location',
 'date',
 'total_cases',
 'new_cases',
 'new_cases_smoothed',
 'total_deaths',
 'new_deaths',
 'new_deaths_smoothed',
 'total_cases_per_million',
 'new_cases_per_million',
 'new_cases_smoothed_per_million',
 'total_deaths_per_million',
 'new_deaths_per_million',
 'new_deaths_smoothed_per_million',
 'reproduction_rate',
 'icu_patients',
 'icu_patients_per_million',
 'hosp_patients',
 'hosp_patients_per_million',
 'weekly_icu_admissions',
 'weekly_icu_admissions_per_million',
 'weekly_hosp_admissions',
 'weekly_hosp_admissions_per_million',
 'new_tests',
 'total_tests',
 'total_tests_per_thousand',
 'new_tests_per_thousand',
 'new_tests_smoothed',
 'new_tests_smoothed_per_thousand',
 'positive_rate',
 'tests_per_case',
 'tests_units',
 'total_vaccinations',
 'people_vaccinated',
 'people_fully_vaccinated',
 'total_boosters',
 'new_vaccinations',
 'new_vaccinations_smoothed',
 'total_vaccinations_per_hundred',
 'people_vaccinated_per_hundred',
 'people_fully_vac

In [3]:
Dataset_1.shape
Dataset_1.isnull().sum()

continent                                    5542
location                                        0
date                                            0
total_cases                                  6226
new_cases                                    6229
                                            ...  
human_development_index                     13954
excess_mortality_cumulative_absolute       117427
excess_mortality_cumulative                117427
excess_mortality                           117427
excess_mortality_cumulative_per_million    117427
Length: 64, dtype: int64

In [4]:
Dataset_1.isnull().sum().sum() # gives us the total number of missing values in the entire dataset

3511143

In [5]:
# checking which values are null in the continent column
# and finding it's equivalent country in the location column
# for that we make only choose the location and continent columns and save them in a separate variable
# then we'll make the location our index and return the index values for the null values in the continent column
# this way we can get the names of the countries for which the continent column has a null value
# then we can replace the null values with the correct continent name given the country name

dt_continent = Dataset_1[['continent', 'location']]
dt_continent.head()

,continent,location
0,Asia,Afghanistan
1,Asia,Afghanistan
2,Asia,Afghanistan
3,Asia,Afghanistan
4,Asia,Afghanistan


In [6]:
dt_continent = dt_continent.set_index(dt_continent['location'])
null_values= dt_continent[dt_continent['continent'].isnull()].index.tolist()
set(null_values)

{'Africa',
 'Asia',
 'Europe',
 'European Union',
 'International',
 'North America',
 'Oceania',
 'South America',
 'World'}

In [7]:
# we see that the rows that has missing values for the continent variable
# have continent names in the location variable
# except for three locations that are named as 'world', 'international' and 'European Union'
# We are going to perform a two step process here
# as we know that there's no such country and continent as 'international', 'world' and 'European Union'
# first we'll replace all the values as 'international' in the location column by 'world' and 'European Union' by 'Europe'
# and then we'll fill all the null values with 'world' in the continent column that has 'world' in the location col

Dataset_1['location'].replace({'International': 'World', 'European Union' : 'Europe'}, inplace=True)
Dataset_1.continent.unique()

array(['Asia', nan, 'Europe', 'Africa', 'North America', 'South America',
       'Oceania'], dtype=object)

In [8]:
Dataset_1['continent'].fillna(Dataset_1['location'], inplace=True)
Dataset_1.continent.unique()

array(['Asia', 'Africa', 'Europe', 'North America', 'South America',
       'Oceania', 'World'], dtype=object)

We see that we don't have any missing values in the continent variable now

In [12]:
#let's see how many unique countries do we have
Dataset_1.location.nunique()

231

In [9]:
Dataset_1.dtypes

continent                                   object
location                                    object
date                                        object
total_cases                                float64
new_cases                                  float64
                                            ...   
human_development_index                    float64
excess_mortality_cumulative_absolute       float64
excess_mortality_cumulative                float64
excess_mortality                           float64
excess_mortality_cumulative_per_million    float64
Length: 64, dtype: object

In [10]:
# the only object types here are continent, location and tests_units
Dataset_1.tests_units.unique()

array([nan, 'tests performed', 'people tested', 'units unclear',
       'samples tested'], dtype=object)

In [11]:
# replace null values in 'tests_units' with 'units unclear'
# 'units unclear' is one of the values in tests_units col

Dataset_1['tests_units'].fillna('units unclear', inplace=True)
Dataset_1.tests_units.unique()

array(['units unclear', 'tests performed', 'people tested',
       'samples tested'], dtype=object)

In [13]:
#sorting the dataset according to the date column

Dataset_1['date'] = pd.to_datetime(Dataset_1['date'])
Dataset_1 = Dataset_1.set_index(Dataset_1['date'])
Dataset_1 = Dataset_1.sort_index()

#creating a new dataframe with the vaccination columns only to see where
#the first non-null value for vaccinations is
new_df = Dataset_1.iloc[:, 33:44]
new_df.head()
list(new_df.columns)
new_df.first_valid_index()

Timestamp('2020-12-01 00:00:00')

In [14]:
#splitting the dataset according to the date we got for the first non null value in the 
#vaccination columns

before_vax = Dataset_1['1/1/2020':'11/30/2020']
after_vax  = Dataset_1['12/1/2020':]

before_vax = pd.DataFrame(before_vax)
after_vax = pd.DataFrame(after_vax)
before_vax.head()

,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
date,,,,,,,,,,,,,,,,,,,,,
2020-01-01,South America,Peru,2020-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.8,NaN,NaN,1.60,76.74,0.777,NaN,NaN,NaN,NaN
2020-01-01,South America,Argentina,2020-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,16.2,27.7,NaN,5.00,76.67,0.845,NaN,NaN,NaN,NaN
2020-01-01,North America,Mexico,2020-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.9,21.4,87.847,1.38,75.05,0.779,NaN,NaN,NaN,NaN
2020-01-02,North America,Mexico,2020-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.9,21.4,87.847,1.38,75.05,0.779,NaN,NaN,NaN,NaN
2020-01-02,South America,Peru,2020-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.8,NaN,NaN,1.60,76.74,0.777,NaN,NaN,NaN,NaN


In [15]:
after_vax.head()

,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
date,,,,,,,,,,,,,,,,,,,,,
2020-12-01,Europe,Austria,2020-12-01,285489.0,3033.0,4397.000,3325.0,141.0,106.857,31569.913,...,28.4,30.9,NaN,7.37,81.54,0.922,NaN,NaN,NaN,NaN
2020-12-01,North America,Belize,2020-12-01,5854.0,0.0,74.143,148.0,0.0,3.286,14457.355,...,NaN,NaN,90.083,1.30,74.62,0.716,NaN,NaN,NaN,NaN
2020-12-01,Africa,Mauritania,2020-12-01,8710.0,109.0,73.857,177.0,0.0,1.143,1824.042,...,NaN,NaN,15.950,NaN,64.92,0.546,NaN,NaN,NaN,NaN
2020-12-01,Asia,Hong Kong,2020-12-01,6396.0,82.0,87.857,109.0,0.0,0.143,846.838,...,NaN,NaN,NaN,NaN,84.86,0.949,NaN,NaN,NaN,NaN
2020-12-01,Asia,Iraq,2020-12-01,554767.0,2218.0,2145.429,12306.0,48.0,39.286,13471.970,...,NaN,NaN,94.576,1.40,70.60,0.674,NaN,NaN,NaN,NaN


In [16]:
#removing vaccination columns from before_vax dataset

before_vax.drop(before_vax.iloc[:, 33:44], axis = 1, inplace = True)
list(before_vax.columns)

['continent',
 'location',
 'date',
 'total_cases',
 'new_cases',
 'new_cases_smoothed',
 'total_deaths',
 'new_deaths',
 'new_deaths_smoothed',
 'total_cases_per_million',
 'new_cases_per_million',
 'new_cases_smoothed_per_million',
 'total_deaths_per_million',
 'new_deaths_per_million',
 'new_deaths_smoothed_per_million',
 'reproduction_rate',
 'icu_patients',
 'icu_patients_per_million',
 'hosp_patients',
 'hosp_patients_per_million',
 'weekly_icu_admissions',
 'weekly_icu_admissions_per_million',
 'weekly_hosp_admissions',
 'weekly_hosp_admissions_per_million',
 'new_tests',
 'total_tests',
 'total_tests_per_thousand',
 'new_tests_per_thousand',
 'new_tests_smoothed',
 'new_tests_smoothed_per_thousand',
 'positive_rate',
 'tests_per_case',
 'tests_units',
 'stringency_index',
 'population',
 'population_density',
 'median_age',
 'aged_65_older',
 'aged_70_older',
 'gdp_per_capita',
 'extreme_poverty',
 'cardiovasc_death_rate',
 'diabetes_prevalence',
 'female_smokers',
 'male_smoke

In [17]:
before_vax.isnull().sum().sum()

1020247

In [18]:
#removing all the null values where there's a null for total cases and replacing the rest with 0s

before_vax.dropna(axis=0, subset=['total_cases'], inplace=True)
before_vax.fillna(0, inplace=True)
before_vax.isnull().sum().sum()

0

In [19]:
after_vax.isnull().sum().sum()

1835181

In [20]:
#removing all the null values where there's a null for total cases and replacing the rest with 0s

after_vax.dropna(axis=0, subset=['total_cases'], inplace=True)
after_vax.fillna(0, inplace=True)
after_vax.isnull().sum().sum()

0

In [21]:
before_vax.shape

(53381, 53)

In [22]:
after_vax.shape

(62137, 64)

Now we'll create dummy variables for the categorical columns in before and after vaccination datasets and save them as seperate datasets.
First we'll create dummy variables for the continent varaible and tests_units removing the location varaibale which has countries as it's values.

In [23]:
# before vaccination 

cont_dummy_before_vax = pd.get_dummies(data= before_vax, columns=['continent', 'tests_units'])

cont_dummy_before_vax.drop(cont_dummy_before_vax.columns[[0]], axis =1, inplace = True)

list(cont_dummy_before_vax.columns)

['date',
 'total_cases',
 'new_cases',
 'new_cases_smoothed',
 'total_deaths',
 'new_deaths',
 'new_deaths_smoothed',
 'total_cases_per_million',
 'new_cases_per_million',
 'new_cases_smoothed_per_million',
 'total_deaths_per_million',
 'new_deaths_per_million',
 'new_deaths_smoothed_per_million',
 'reproduction_rate',
 'icu_patients',
 'icu_patients_per_million',
 'hosp_patients',
 'hosp_patients_per_million',
 'weekly_icu_admissions',
 'weekly_icu_admissions_per_million',
 'weekly_hosp_admissions',
 'weekly_hosp_admissions_per_million',
 'new_tests',
 'total_tests',
 'total_tests_per_thousand',
 'new_tests_per_thousand',
 'new_tests_smoothed',
 'new_tests_smoothed_per_thousand',
 'positive_rate',
 'tests_per_case',
 'stringency_index',
 'population',
 'population_density',
 'median_age',
 'aged_65_older',
 'aged_70_older',
 'gdp_per_capita',
 'extreme_poverty',
 'cardiovasc_death_rate',
 'diabetes_prevalence',
 'female_smokers',
 'male_smokers',
 'handwashing_facilities',
 'hospital_

In [24]:
# after vaccination
cont_dummy_after_vax = pd.get_dummies(data= after_vax, columns=['continent', 'tests_units'])

cont_dummy_after_vax.drop(cont_dummy_after_vax.columns[[0]], axis =1, inplace = True)

list(cont_dummy_after_vax.columns)

['date',
 'total_cases',
 'new_cases',
 'new_cases_smoothed',
 'total_deaths',
 'new_deaths',
 'new_deaths_smoothed',
 'total_cases_per_million',
 'new_cases_per_million',
 'new_cases_smoothed_per_million',
 'total_deaths_per_million',
 'new_deaths_per_million',
 'new_deaths_smoothed_per_million',
 'reproduction_rate',
 'icu_patients',
 'icu_patients_per_million',
 'hosp_patients',
 'hosp_patients_per_million',
 'weekly_icu_admissions',
 'weekly_icu_admissions_per_million',
 'weekly_hosp_admissions',
 'weekly_hosp_admissions_per_million',
 'new_tests',
 'total_tests',
 'total_tests_per_thousand',
 'new_tests_per_thousand',
 'new_tests_smoothed',
 'new_tests_smoothed_per_thousand',
 'positive_rate',
 'tests_per_case',
 'total_vaccinations',
 'people_vaccinated',
 'people_fully_vaccinated',
 'total_boosters',
 'new_vaccinations',
 'new_vaccinations_smoothed',
 'total_vaccinations_per_hundred',
 'people_vaccinated_per_hundred',
 'people_fully_vaccinated_per_hundred',
 'total_boosters_per_

In [25]:
#################### CREATING DUMMIES FOR COUNTRIES##########################

# before vaccination 

country_dummy_before_vax = pd.get_dummies(data= before_vax, columns=['location', 'tests_units'])

country_dummy_before_vax.drop(country_dummy_before_vax.columns[[0]], axis =1, inplace = True)

list(country_dummy_before_vax.columns)

['date',
 'total_cases',
 'new_cases',
 'new_cases_smoothed',
 'total_deaths',
 'new_deaths',
 'new_deaths_smoothed',
 'total_cases_per_million',
 'new_cases_per_million',
 'new_cases_smoothed_per_million',
 'total_deaths_per_million',
 'new_deaths_per_million',
 'new_deaths_smoothed_per_million',
 'reproduction_rate',
 'icu_patients',
 'icu_patients_per_million',
 'hosp_patients',
 'hosp_patients_per_million',
 'weekly_icu_admissions',
 'weekly_icu_admissions_per_million',
 'weekly_hosp_admissions',
 'weekly_hosp_admissions_per_million',
 'new_tests',
 'total_tests',
 'total_tests_per_thousand',
 'new_tests_per_thousand',
 'new_tests_smoothed',
 'new_tests_smoothed_per_thousand',
 'positive_rate',
 'tests_per_case',
 'stringency_index',
 'population',
 'population_density',
 'median_age',
 'aged_65_older',
 'aged_70_older',
 'gdp_per_capita',
 'extreme_poverty',
 'cardiovasc_death_rate',
 'diabetes_prevalence',
 'female_smokers',
 'male_smokers',
 'handwashing_facilities',
 'hospital_

In [26]:
# after vaccination
country_dummy_after_vax = pd.get_dummies(data= after_vax, columns=['location', 'tests_units'])

country_dummy_after_vax.drop(country_dummy_after_vax.columns[[0]], axis =1, inplace = True)
list(country_dummy_after_vax.columns)

['date',
 'total_cases',
 'new_cases',
 'new_cases_smoothed',
 'total_deaths',
 'new_deaths',
 'new_deaths_smoothed',
 'total_cases_per_million',
 'new_cases_per_million',
 'new_cases_smoothed_per_million',
 'total_deaths_per_million',
 'new_deaths_per_million',
 'new_deaths_smoothed_per_million',
 'reproduction_rate',
 'icu_patients',
 'icu_patients_per_million',
 'hosp_patients',
 'hosp_patients_per_million',
 'weekly_icu_admissions',
 'weekly_icu_admissions_per_million',
 'weekly_hosp_admissions',
 'weekly_hosp_admissions_per_million',
 'new_tests',
 'total_tests',
 'total_tests_per_thousand',
 'new_tests_per_thousand',
 'new_tests_smoothed',
 'new_tests_smoothed_per_thousand',
 'positive_rate',
 'tests_per_case',
 'total_vaccinations',
 'people_vaccinated',
 'people_fully_vaccinated',
 'total_boosters',
 'new_vaccinations',
 'new_vaccinations_smoothed',
 'total_vaccinations_per_hundred',
 'people_vaccinated_per_hundred',
 'people_fully_vaccinated_per_hundred',
 'total_boosters_per_

In [27]:
# Cleaning Dataset-3

Dataset_3 = pd.read_csv(r'C:\Users\burq_\Downloads\DAAN 888-Design and Implementation of Analytics system\DAAN 888 Datasets\Final datasets\Dataset-3 COVID-19_Vaccinations_in_the_United_States_County.csv')

list(Dataset_3.columns)


['Date',
 'FIPS',
 'MMWR_week',
 'Recip_County',
 'Recip_State',
 'Series_Complete_Pop_Pct',
 'Series_Complete_Yes',
 'Series_Complete_12Plus',
 'Series_Complete_12PlusPop_Pct',
 'Series_Complete_18Plus',
 'Series_Complete_18PlusPop_Pct',
 'Series_Complete_65Plus',
 'Series_Complete_65PlusPop_Pct',
 'Completeness_pct',
 'Administered_Dose1_Recip',
 'Administered_Dose1_Pop_Pct',
 'Administered_Dose1_Recip_12Plus',
 'Administered_Dose1_Recip_12PlusPop_Pct',
 'Administered_Dose1_Recip_18Plus',
 'Administered_Dose1_Recip_18PlusPop_Pct',
 'Administered_Dose1_Recip_65Plus',
 'Administered_Dose1_Recip_65PlusPop_Pct',
 'SVI_CTGY',
 'Series_Complete_Pop_Pct_SVI',
 'Series_Complete_12PlusPop_Pct_SVI',
 'Series_Complete_18PlusPop_Pct_SVI',
 'Series_Complete_65PlusPop_Pct_SVI',
 'Metro_status',
 'Series_Complete_Pop_Pct_UR_Equity',
 'Series_Complete_12PlusPop_Pct_UR_Equity',
 'Series_Complete_18PlusPop_Pct_UR_Equity',
 'Series_Complete_65PlusPop_Pct_UR_Equity']

In [28]:
Dataset_3.drop(Dataset_3.columns[[1,3,6,7,9,11,14,16,18,20,22,23,24,25,26,27,28,29,30,31]], axis=1, inplace=True)

list(Dataset_3.columns)

['Date',
 'MMWR_week',
 'Recip_State',
 'Series_Complete_Pop_Pct',
 'Series_Complete_12PlusPop_Pct',
 'Series_Complete_18PlusPop_Pct',
 'Series_Complete_65PlusPop_Pct',
 'Completeness_pct',
 'Administered_Dose1_Pop_Pct',
 'Administered_Dose1_Recip_12PlusPop_Pct',
 'Administered_Dose1_Recip_18PlusPop_Pct',
 'Administered_Dose1_Recip_65PlusPop_Pct']

In [29]:
Dataset_3.dtypes

Date                                       object
MMWR_week                                   int64
Recip_State                                object
Series_Complete_Pop_Pct                   float64
Series_Complete_12PlusPop_Pct             float64
Series_Complete_18PlusPop_Pct             float64
Series_Complete_65PlusPop_Pct             float64
Completeness_pct                          float64
Administered_Dose1_Pop_Pct                float64
Administered_Dose1_Recip_12PlusPop_Pct    float64
Administered_Dose1_Recip_18PlusPop_Pct    float64
Administered_Dose1_Recip_65PlusPop_Pct    float64
dtype: object

In [30]:
#checking the states that are in the dataset and the number of records each has
Dataset_3['Recip_State'].value_counts()

TX     78496
GA     49444
VA     41419
KY     37394
MO     35853
KS     32759
IL     31829
NC     31213
IA     30905
TN     29666
NE     29047
IN     28740
OH     27503
MN     27204
MI     25960
MS     25651
PR     24409
OK     24103
AR     23485
WI     22562
PA     21014
AL     21013
FL     21013
SD     20705
CO     20088
LA     20084
NY     19468
CA     18234
MT     17611
WV     17303
ND     16687
SC     14523
ID     13906
WA     12362
OR     11433
NM     10505
UT      9269
AK      9268
MD      7723
WY      7416
NJ      6796
NV      5560
ME      5250
AZ      4941
MA      4635
VT      4633
NH      3396
CT      2778
RI      1851
HI      1545
DE      1233
VI      1228
DC       614
GU       613
FM       305
UNK      304
MP       302
AS       300
PW       299
MH       292
Name: Recip_State, dtype: int64

In [31]:
Dataset_3.shape

(1014142, 12)

In [32]:
Dataset_3.isnull().sum()

Date                                          0
MMWR_week                                     0
Recip_State                                   0
Series_Complete_Pop_Pct                       0
Series_Complete_12PlusPop_Pct             13906
Series_Complete_18PlusPop_Pct                 0
Series_Complete_65PlusPop_Pct                 0
Completeness_pct                              0
Administered_Dose1_Pop_Pct                    0
Administered_Dose1_Recip_12PlusPop_Pct    13906
Administered_Dose1_Recip_18PlusPop_Pct        0
Administered_Dose1_Recip_65PlusPop_Pct        0
dtype: int64

In [33]:
# we only have null values in the columns for 12+ vaccination 
# That's probably because the vaccination became available later for kids age 12 and up
# we will impute it with 0

Dataset_3.fillna(0, inplace = True)

In [34]:
Dataset_3.isnull().sum()

Date                                      0
MMWR_week                                 0
Recip_State                               0
Series_Complete_Pop_Pct                   0
Series_Complete_12PlusPop_Pct             0
Series_Complete_18PlusPop_Pct             0
Series_Complete_65PlusPop_Pct             0
Completeness_pct                          0
Administered_Dose1_Pop_Pct                0
Administered_Dose1_Recip_12PlusPop_Pct    0
Administered_Dose1_Recip_18PlusPop_Pct    0
Administered_Dose1_Recip_65PlusPop_Pct    0
dtype: int64

In [9]:
#Cleaning dataset 5

Dataset_5 = pd.read_csv(r'C:\Users\burq_\Downloads\DAAN 888-Design and Implementation of Analytics system\DAAN 888 Datasets\Final datasets\Dataset-5 COVID-19_Case_Surveillance_Public_Use_Data_with_Geography.csv')

list(Dataset_5.columns)

['case_month',
 'res_state',
 'state_fips_code',
 'res_county',
 'county_fips_code',
 'age_group',
 'sex',
 'race',
 'ethnicity',
 'case_positive_specimen_interval',
 'case_onset_interval',
 'process',
 'exposure_yn',
 'current_status',
 'symptom_status',
 'hosp_yn',
 'icu_yn',
 'death_yn',
 'underlying_conditions_yn']

In [ ]:
Dataset_5.replace(['Missing', 'Unknown'], 'not specified', inplace=True)

In [10]:
Dataset_5.isnull().sum()

case_month                               13
res_state                               746
state_fips_code                         746
res_county                          1658731
county_fips_code                    1658731
age_group                            356631
sex                                  943678
race                                5365025
ethnicity                           6293242
case_positive_specimen_interval    21118524
case_onset_interval                16011783
process                                   0
exposure_yn                               0
current_status                            0
symptom_status                            0
hosp_yn                                   0
icu_yn                                    0
death_yn                            1490254
underlying_conditions_yn           27956615
dtype: int64

In [11]:
Dataset_5.dtypes

case_month                          object
res_state                           object
state_fips_code                    float64
res_county                          object
county_fips_code                   float64
age_group                           object
sex                                 object
race                                object
ethnicity                           object
case_positive_specimen_interval    float64
case_onset_interval                float64
process                             object
exposure_yn                         object
current_status                      object
symptom_status                      object
hosp_yn                             object
icu_yn                              object
death_yn                            object
underlying_conditions_yn            object
dtype: object

In [12]:
Dataset_5a = Dataset_5.drop(Dataset_5.columns[[2,3,4,9,10,11]], axis=1)
list(Dataset_5a.columns)

['case_month',
 'res_state',
 'age_group',
 'sex',
 'race',
 'ethnicity',
 'exposure_yn',
 'current_status',
 'symptom_status',
 'hosp_yn',
 'icu_yn',
 'death_yn',
 'underlying_conditions_yn']

In [46]:
Dataset_5a.dtypes

case_month                  object
res_state                   object
age_group                   object
sex                         object
race                        object
ethnicity                   object
exposure_yn                 object
current_status              object
symptom_status              object
hosp_yn                     object
icu_yn                      object
death_yn                    object
underlying_conditions_yn    object
dtype: object

In [48]:
# gives us the count of unique values in each column
Dataset_5a.apply(pd.value_counts)

,case_month,res_state,age_group,sex,race,ethnicity,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
0 - 17 years,NaN,NaN,3760274.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,NaN,1039.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18 to 49 years,NaN,NaN,15879682.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01,4183.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-02,4841.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
WV,NaN,9796.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WY,NaN,1295.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
White,NaN,NaN,NaN,NaN,13119503.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Yes,NaN,NaN,NaN,NaN,NaN,NaN,1991992.0,NaN,NaN,1408341.0,126677.0,291255.0,1860547.0


In [13]:
# lets check a few columns that we are interested in individually
Dataset_5a['res_state'].value_counts()

CA    4263508
NY    2296687
FL    1944432
IL    1503858
PA    1260445
OH    1164869
NC    1134556
NJ    1101062
GA    1024190
AZ     977442
TN     951324
IN     810952
MA     742329
VA     728276
MI     699252
SC     679150
MN     632586
AL     626132
WI     613109
CO     589381
WA     522954
OK     509860
MD     489767
UT     452890
IA     398813
NV     353700
KS     353256
CT     347529
KY     334023
AR     318004
OR     249940
NM     223703
ID     211699
NE     200771
RI     146408
MT     120472
SD     113732
DE     111450
NH      85260
TX      82000
AK      77725
ME      72245
MO      60394
MS      53876
DC      52323
HI      36659
PR      33407
VT      24936
ND      24174
LA      19959
WV       9796
GU       8190
VI       4915
WY       1295
18       1039
Name: res_state, dtype: int64

In [50]:
Dataset_5a['age_group'].value_counts()

18 to 49 years    15879682
50 to 64 years     5801945
65+ years          3796330
0 - 17 years       3760274
not specified       356631
Missing             256588
Name: age_group, dtype: int64

In [14]:
# changing missing to not specified

Dataset_5a['age_group'].replace({'Missing' : 'not specified'}, inplace=True)

In [15]:
Dataset_5a['age_group'].value_counts()

18 to 49 years    15879682
50 to 64 years     5801945
65+ years          3796330
0 - 17 years       3760274
not specified       256588
Name: age_group, dtype: int64

In [16]:
# As all the remaining columns are categorical, we are going to fill the null values with 'not specified' instead of 0
Dataset_5a.fillna('not specified', inplace = True)

In [17]:
Dataset_5a.isnull().sum()

case_month                  0
res_state                   0
age_group                   0
sex                         0
race                        0
ethnicity                   0
exposure_yn                 0
current_status              0
symptom_status              0
hosp_yn                     0
icu_yn                      0
death_yn                    0
underlying_conditions_yn    0
dtype: int64

In [18]:
#removing underlying conditions from dataset_5a

Dataset_5b = Dataset_5a.drop(Dataset_5a.columns[[-1]], axis=1)
list(Dataset_5b.columns)

['case_month',
 'res_state',
 'age_group',
 'sex',
 'race',
 'ethnicity',
 'exposure_yn',
 'current_status',
 'symptom_status',
 'hosp_yn',
 'icu_yn',
 'death_yn']

In [19]:
# dropping cols from dataset_5 and then dropping the null values

Dataset_5c = Dataset_5.drop(Dataset_5.columns[[2,3,4,9,10,11,18]], axis =1)
list(Dataset_5c.columns)


['case_month',
 'res_state',
 'age_group',
 'sex',
 'race',
 'ethnicity',
 'exposure_yn',
 'current_status',
 'symptom_status',
 'hosp_yn',
 'icu_yn',
 'death_yn']

In [20]:
Dataset_5c.dropna(inplace = True)
Dataset_5c.isnull().sum()

case_month        0
res_state         0
age_group         0
sex               0
race              0
ethnicity         0
exposure_yn       0
current_status    0
symptom_status    0
hosp_yn           0
icu_yn            0
death_yn          0
dtype: int64

In [21]:
Dataset_5a.shape

(29851450, 13)

In [22]:
Dataset_5b.shape

(29851450, 12)

In [23]:
Dataset_5c.shape

(22758349, 12)